# Geographic Heat Map of COVID Cases by MODZCTA
This notebook creates a geographic heat map of COVID cases by [MODZCTA](https://github.com/nychealth/coronavirus-data#geography-zip-codes-and-zctas). This requires three open data sets:

* Total COVID data by MODZCTA
* Latest COVID data by MODZCTA
* GeoJSON of polygons of the ZIPCODES of NYC

## Fetch and Process Total COVID data by MODZCTA
This data set provides clean total data to-date. It also provides population data. We can use this data set to build maps by total case rates and death rates if we desire (_case rate_ and _death rate_ data is normalised per 100K people).

In [1]:
# Fetch total to-date data on COVID by MODZCTA
import pandas as pd
URL = "https://github.com/nychealth/coronavirus-data/blob/master/totals/data-by-modzcta.csv"
raw_totals_df = pd.read_csv(URL + "?raw=true")
print("Fetched total data from %d MODZCTAs" % len(raw_totals_df))

Fetched total data from 177 MODZCTAs


In [2]:
# Standardize the data set, show the worst five areas by case_rate
df_totals = raw_totals_df[['MODIFIED_ZCTA', 'NEIGHBORHOOD_NAME', 
                           'BOROUGH_GROUP', 'POP_DENOMINATOR',
                           'COVID_CASE_RATE',  'COVID_DEATH_RATE' ]]
df_totals = df_totals.rename(columns = {
    'MODIFIED_ZCTA' : 'zip_code', 
    'NEIGHBORHOOD_NAME' : 'neighborhood', 
    'BOROUGH_GROUP' : 'borough', 
    'POP_DENOMINATOR' : 'population', 
    'COVID_CASE_RATE' : 'total_case_rate',  # Per 100K poeple
    'COVID_DEATH_RATE' : 'total_death_rate' # Per 100K people
})
df_totals.zip_code = df_totals.zip_code.astype(str)
df_totals.population = df_totals.population.astype(float)
df_totals.total_case_rate = df_totals.total_case_rate.astype(float)
df_totals.total_death_rate = df_totals.total_death_rate.astype(float)
df_totals['total_case_rank'] = df_totals['total_case_rate'].rank()
df_totals['total_death_rank'] = df_totals['total_death_rate'].rank()
df_totals.sort_values('total_case_rate', ascending=False).head(5)

,zip_code,neighborhood,borough,population,total_case_rate,total_death_rate,total_case_rank,total_death_rank
176,11697,Breezy Point,Queens,3392.62,16830.67,383.19,177.0,128.0
49,10306,Lighthouse Hill/Midland Beach/New Dorp/Oakwood,Staten Island,52806.81,14464.04,433.66,176.0,153.0
52,10309,Charleston/Prince's Bay/Woodrow,Staten Island,32939.56,14086.40,203.40,175.0,38.0
140,11369,Airport/East Elmhurst,Queens,33627.74,13836.79,636.38,174.0,172.0
50,10307,Tottenville,Staten Island,14912.16,13673.41,187.77,173.0,34.0


## Fetch, Process, and Mege the Latest Case Rate Data
This data set provides the latest case rates by **Rolling 7-Days**. This is useful as it removed the effect of any given day of the week (i.e., lower rates on Sundays.) These are total rates. As such, we need to normalise the data per 100K people (before we do any ranking).

In [3]:
# Fetch latest data on COVID by MODXCTA
URL = "https://github.com/nychealth/coronavirus-data/blob/master/latest/last7days-by-modzcta.csv"
raw_new_df = pd.read_csv(URL + "?raw=true")
print("Fetched lastest data from %d MODZCTAs" % len(raw_new_df))

Fetched lastest data from 177 MODZCTAs


In [4]:
# Standardize the data set, show the worst five areas by total new rolling case count
df_new = raw_new_df[['modzcta', 'people_positive']]
df_new = df_new.rename(columns = {
    'modzcta' : 'zip_code', 
    'people_positive' : 'new_cases_7d' # Total cases, rolling 7-days
})
df_new.zip_code = df_new.zip_code.astype(str)
df_new.new_cases_7d = df_new.new_cases_7d.astype(float)
df_new.sort_values('new_cases_7d', ascending=False).head(5)

,zip_code,new_cases_7d
139,11368,26.0
95,11208,22.0
58,10453,19.0
145,11375,18.0
54,10312,17.0


In [5]:
# Merge the data sets, showing the worst death rates 
df = pd.merge(df_totals, df_new, on='zip_code')
df.sort_values('total_death_rate', ascending=False).head(5)

,zip_code,neighborhood,borough,population,total_case_rate,total_death_rate,total_case_rank,total_death_rank,new_cases_7d
125,11239,East New York,Brooklyn,13022.30,12770.40,944.53,166.0,177.0,3.0
111,11224,Brighton Beach/Coney Island/Seagate,Brooklyn,46270.81,12312.30,758.58,163.0,176.0,7.0
126,11354,Flushing/Murray Hill,Queens,53918.85,9024.67,752.98,93.0,175.0,3.0
121,11235,Brighton Beach/Manhattan Beach/Sheepshead Bay,Brooklyn,78197.62,13065.61,694.39,169.0,174.0,9.0
172,11691,Edgemere/Far Rockaway,Queens,66856.31,11434.97,689.54,151.0,173.0,10.0


In [6]:
# Normalise the new case rate so we can compare geographies and rank them, show the worst 5
df['new_case_rate_7d'] = (df['new_cases_7d'] / df['population'] * 100000.0).round(decimals=2)
df['new_case_rank'] = df['new_case_rate_7d'].rank()
df.sort_values('new_case_rate_7d', ascending=False).head(5)

,zip_code,neighborhood,borough,population,total_case_rate,total_death_rate,total_case_rank,total_death_rank,new_cases_7d,new_case_rate_7d,new_case_rank
32,10036,Hell's Kitchen/Midtown Manhattan,Manhattan,27242.15,7014.86,209.23,45.0,41.0,13.0,47.72,177.0
171,11436,South Jamaica/South Ozone Park,Queens,20042.54,9779.20,339.28,113.0,105.0,9.0,44.90,176.0
59,10454,Mott Haven/Port Morris,Bronx,37829.72,10531.40,428.23,128.0,149.0,14.0,37.01,175.0
88,11109,Long Island City,Queens,5762.76,6403.19,52.06,39.0,5.0,2.0,34.71,174.0
3,10004,Financial District,Manhattan,2972.12,8546.10,67.29,78.0,8.0,1.0,33.65,173.0


We can already see that worst places for new cases are NOT the worst places for total cases. At least one looks like a place where lots of tourists come (i.e., Hell's Kitchen and Times Square)😟
## Load and Process the GeoJSON

In [7]:
# Get the GeoJSON
import requests
import json

GEO_URL = "https://raw.githubusercontent.com/fedhere/PUI2015_EC/master/mam1612_EC/nyc-zip-code-tabulation-areas-polygons.geojson"
resp = requests.get(GEO_URL)
zipcodes_json = json.loads(resp.text)
print("Fetched set of %d GeoJSON features (this is more than just MODZCTA data)" % len(zipcodes_json['features']))

Fetched set of 262 GeoJSON features (this is more than just MODZCTA data)


In [8]:
# Now build a dictionary from our dataframe, so we can look up COVID data by GeoJSON feature
# Show the entrye Hell's Kitchen and Time Square 
by_zip = df.set_index('zip_code').T.to_dict()
by_zip['10036']

{'neighborhood': "Hell's Kitchen/Midtown Manhattan",
 'borough': 'Manhattan',
 'population': 27242.15,
 'total_case_rate': 7014.86,
 'total_death_rate': 209.23,
 'total_case_rank': 45.0,
 'total_death_rank': 41.0,
 'new_cases_7d': 13.0,
 'new_case_rate_7d': 47.72,
 'new_case_rank': 177.0}

Now we need we need to append the GeoJSON dictionary with COVID data. We will do this using a functional programming approach. Yes, it is more memory intensive. But if I screw it up, I can go back to my original data.

In [9]:
# Extract the original features
original_features = zipcodes_json['features']

# Build a new list of amended features, iterating through the orginal, then amending and apending
new_features = list()
for feature in original_features:
    properties = feature['properties']
    modcta = properties['postalCode']
    
    # Use a try/except clause to cover zip coder where we have no data (e.g, Central Park)
    try: 
        data = by_zip[modcta]
        neighborhood = data['neighborhood']
        population = data['population']
        new_cases_7d = data['new_cases_7d']
        new_case_rate_7d = data['new_case_rate_7d']
        new_case_rank = data['new_case_rank']
        total_case_rate = data['total_case_rate']
        total_case_rank = data['total_case_rank']
        total_death_rate = data['total_death_rate']
        total_death_rank = data['total_death_rank']
    except:
        neighborhood = population = 'N/A'
        new_cases_7d = new_case_rate_7d = new_case_rank = 'N/A' 
        total_case_rate = total_case_rank = 'N/A' 
        total_death_rate = total_death_rank = 'N/A'
    
    properties['neighborhood'] = neighborhood
    properties['population'] = population
    properties['new_cases_7d'] = new_cases_7d
    properties['new_case_rate_7d'] = new_case_rate_7d
    properties['new_case_rank'] = new_case_rank
    properties['total_case_rate'] = total_case_rate
    properties['total_case_rank'] = total_case_rank
    properties['total_death_rate'] = total_death_rate
    properties['total_death_rank'] = total_death_rank
 
    feature['properties'] = properties
    new_features.append(feature)

# Now assemble a new GeoJSON dictionary, assigning the amended feature set
new_geojson = zipcodes_json
new_geojson['features'] = new_features

# Now show Hell's Kitchen
import json
for feature in new_features:
    if feature['properties']['postalCode'] == '10036':
        print(json.dumps(feature, indent=2))

{
  "type": "Feature",
  "id": 105,
  "properties": {
    "OBJECTID": 106,
    "postalCode": "10036",
    "PO_NAME": "New York",
    "STATE": "NY",
    "borough": "Manhattan",
    "ST_FIPS": "36",
    "CTY_FIPS": "061",
    "BLDGpostalCode": 0,
    "Shape_Leng": 16418.6066682,
    "Shape_Area": 11395110.5096,
    "@id": "http://nyc.pediacities.com/Resource/PostalCode/10036",
    "neighborhood": "Hell's Kitchen/Midtown Manhattan",
    "population": 27242.15,
    "new_cases_7d": 13.0,
    "new_case_rate_7d": 47.72,
    "new_case_rank": 177.0,
    "total_case_rate": 7014.86,
    "total_case_rank": 45.0,
    "total_death_rate": 209.23,
    "total_death_rank": 41.0
  },
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -73.98134106844775,
          40.758645323851304
        ],
        [
          -73.97811606058137,
          40.7572844526636
        ],
        [
          -73.98088709157031,
          40.753481009691896
        ],
        [
          

## Let's Build Some Maps
In theory, we can now build a Folium map. We will:
* Centre the map on NYC (with the corect zoom level)
* Overlay the zip code polygons
* Then build a data interface layer of tool tips that shows data on each zip code

This took several iterations to develop. The code below is a "clean" implementation:

In [10]:
# Imports for general mapping
import numpy as np
import folium
import matplotlib as plt

def build_heat_map(color_by):
    """
    This function returns a color-coded GeoJSON "heat map" of COVID data
    
    Input Parameter: 
    color_by : What we want to color the map by (most red is worst). Options:
        'new_cases' : Count of new cases (7d)
        'new_case_rate' : Rate of new cases per 100K people (7d)
        'total_case_rate' : Rate of total cases per 100K people (to-date)
        'total_death_rate' : Rate of total deaths per 100K people (to-date)
        
    Returns: Folium map object (Choropleth by geo_data)
    """

    if color_by == 'new_cases':
        print("Creating map by count of new COVID cases (7d)")
        legend_title = 'New COVID Case Count (Rolling 7-Days)'
        color_code_by = 'new_cases_7d'
        
    elif color_by == 'new_case_rate':
        print("Creating map by count of new COVID cases (7d)")
        legend_title = 'New COVID Case Rate per 100K People (Rolling 7-Days)'
        color_code_by = 'new_case_rate_7d'   
        
    elif color_by == 'total_death_rate':
        print("Creating map by total COVID death rate per 100K People (to-date)")
        legend_title = 'COVID Death Rate per 100K People (To-date)'
        color_code_by = 'total_death_rate'
        
    else:
        if color_by != 'total_case_rate':
            print("Options are: 'new_cases', 'new_case_rate', 'total_case_rate', total_death_rate'")
        print("Creating map by total COVID case rate per 100K People (to-date)")
        legend_title = 'COVID Case Rate per 100K People (To-date)'
        color_code_by = 'total_case_rate' 
    
    # Build the map, centred around NYC
    map = folium.Map(location=[40.63, -73.95], zoom_start=11)
    
    # Add the GeoJSON defining the polygons for each zip code
    folium.Choropleth(
        geo_data=new_geojson,
        data=df,
        columns=['zip_code', color_code_by],
        key_on='feature.properties.postalCode',
        fill_color='YlOrRd',
        nan_fill_color='gray',
        legend_name=legend_title,
        highlight=True
    ).add_to(map)
 
    # Now build the styling functions for the tool tips
    style_function = lambda x: {'fillColor': '#ffffff', 
                                'color':'#000000', 
                                'fillOpacity': 0.1, 
                                'weight': 0.1}
    highlight_function = lambda x: {'fillColor': '#000000', 
                                    'color':'#000000', 
                                    'fillOpacity': 0.50, 
                                    'weight': 0.1}

    # Now build the network interface layer (i.e., the tool tips) using the amended data set
    NIL = folium.features.GeoJson(
        data=new_geojson,
        style_function=style_function, 
        control=False,
        highlight_function=highlight_function, 
        tooltip=folium.features.GeoJsonTooltip(
            fields=['postalCode', 'neighborhood', 
                    'new_cases_7d', 'new_case_rate_7d', 'new_case_rank',
                    'total_case_rate', 'total_case_rank',
                    'total_death_rate', 'total_death_rank'],
            aliases=['Postal Code: ','Neighborhood: ',
                     'New Case Count (7d):', 
                     'New Cases per 100K People (7d): ', 'New Case Rank (7d): ',
                     'Total Cases per 100K People: ', 'Total Case Rank: ',
                     'Total Deaths per 100K People: ', 'Total Death Rank: '],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
        )
    )

    # Now add the NIL to the map and display it
    map.add_child(NIL)
    map.keep_in_front(NIL)
    folium.LayerControl().add_to(map)
    return(map)

# Force the error to show the options, and display by case rate by date
build_heat_map('x')

Options are: 'new_cases', 'new_case_rate', 'total_case_rate', total_death_rate'
Creating map by total COVID case rate per 100K People (to-date)


In [11]:
# Now show a map of the new case rate (7d) per 100K people
map = build_heat_map('new_case_rate')
map

Creating map by count of new COVID cases (7d)


Now we can export the data to an HTML file, then re-open in a new brower tab. This will let us explore the map much more interactively

In [12]:
FILENAME = 'nyc_covid_heat_map.html'
map.save(FILENAME)

import webbrowser, os
webbrowser.open('file://' + os.path.realpath(FILENAME))

True